In [3]:
# === Dataset source: merged CSV (Jupyter & Streamlit compatible) ===
import altair as alt
import re
from pathlib import Path
import pandas as pd, numpy as np
import streamlit as st

# 1) BASE 경로: .py 파일이면 __file__, 노트북이면 CWD 사용
try:
    BASE = Path(__file__).parent
except NameError:
    BASE = Path.cwd()

# 2) 후보 경로에서 CSV 검색 (존재하는 첫 파일 사용)
CANDIDATES = [
    BASE / "spotify_merged.csv",                          # 권장: 루트에 두기
    BASE / "data" / "raw" / "spotify_merged.csv",
    BASE / "spotify_cleaned_final_v2.csv",                # 백업: 정제 CSV (있다면)
    BASE / "data" / "raw" / "spotify_cleaned_final_v2.csv",
]
CSV_PATH = next((p for p in CANDIDATES if p.exists()), None)
if CSV_PATH is None:
    st.error("❌ merged CSV를 찾지 못했습니다. `spotify_merged.csv`를 프로젝트 루트(또는 data/raw)에 두세요.")
    st.stop()

@st.cache_data(show_spinner=False)
def load_merged_csv(path: Path):
    """원본+매출지표 병합 CSV 로더
    - df_raw: 원본 그대로 (KPI/프리뷰/결측치 표시에 사용)
    - df: 차트용 작업본 (원본 열수에 영향 없이 파생만 추가)
    """
    df_raw = pd.read_csv(path)

    # 작업본 복사
    dfx = df_raw.copy()

    # revenue_num: 문자→숫자 (₩, 콤마 제거)
    if "revenue" in dfx.columns:
        def to_number(x):
            if pd.isna(x): return np.nan
            s = re.sub(r"[^0-9.\-]", "", str(x))
            return float(s) if s else np.nan
        dfx["revenue_num"] = dfx["revenue"].map(to_number)

    # 요금제 표준화 컬럼
    plan_col = next((c for c in ["subscription_plan","spotify_subscription_plan","subscription_plan_norm"]
                     if c in dfx.columns), None)
    if plan_col and "subscription_plan_norm" not in dfx.columns:
        dfx["subscription_plan_norm"] = dfx[plan_col].astype(str)

    # month 문자열화(Altair 정렬 일관성)
    if "month" in dfx.columns:
        dfx["month"] = dfx["month"].astype(str)

    return df_raw, dfx

# 3) 실제 로드 (여기서 반드시 df_raw/df를 생성해야 아래에서 사용 가능)
try:
    df_raw, df = load_merged_csv(CSV_PATH)
except Exception as e:
    st.error(f"데이터 로드 실패: {e}")
    st.stop()

# 4) 디버깅/확인용 캡션 (원하면 지워도 됨)
st.caption(f"✅ CSV 로드: {CSV_PATH.name} — {len(df_raw):,}행 × {len(df_raw.columns)}열 (원본 기준)")

2025-10-25 22:03:36.704 No runtime found, using MemoryCacheStorageManager
2025-10-25 22:03:36.712 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 22:03:36.712 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 22:03:36.713 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()